In [1]:
import os # Leer imagenes
import torch # Armar modelos
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import albumentations as A # Augmentaciones
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder # Convertir los nombres de las clases en nums
from tqdm import tqdm # Barra de proceso 
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report # Levantar metricas
import io # Python basico

c:\Users\santi\AppData\Local\Programs\Python\Python313\Lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.7'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
import mlflow # Training Tracking and Comparision
import mlflow.pytorch

In [3]:
mlflow.set_experiment("MLP_Clasificador_Imagenes")

<Experiment: artifact_location='file:///c:/Users/santi/OneDrive/Escritorio/skin_dataset_clasification-AGOSTI/mlruns/618597791685242834', creation_time=1749738080759, experiment_id='618597791685242834', last_update_time=1749738080759, lifecycle_stage='active', name='MLP_Clasificador_Imagenes', tags={}>

In [4]:
from torch.utils.tensorboard import SummaryWriter
import torchvision.utils as vutils

In [5]:
# Función para loguear una figura matplotlib en TensorBoard
def plot_to_tensorboard(fig, writer, tag, step):
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    image = Image.open(buf).convert("RGB")
    image = np.array(image)
    image = torch.tensor(image).permute(2, 0, 1) / 255.0
    writer.add_image(tag, image, global_step=step)
    plt.close(fig)

In [6]:
# Función para matriz de confusión y clasificación
def log_classification_report(model, loader, writer, step, prefix="val"):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    fig_cm, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=train_dataset.label_encoder.classes_)
    disp.plot(ax=ax, cmap='Blues', xticks_rotation=45)
    ax.set_title(f'{prefix.title()} - Confusion Matrix')

    # Guardar localmente y subir a MLflow
    fig_path = f"confusion_matrix_{prefix}_epoch_{step}.png"
    fig_cm.savefig(fig_path)
    mlflow.log_artifact(fig_path)
    os.remove(fig_path)

    plot_to_tensorboard(fig_cm, writer, f"{prefix}/confusion_matrix", step)

    cls_report = classification_report(all_labels, all_preds, target_names=train_dataset.label_encoder.classes_)
    writer.add_text(f"{prefix}/classification_report", f"<pre>{cls_report}</pre>", step)

    # También loguear texto del reporte
    with open(f"classification_report_{prefix}_epoch_{step}.txt", "w") as f:
        f.write(cls_report)
    mlflow.log_artifact(f.name)
    os.remove(f.name)


In [7]:

# Crear directorio de logs
run_name = f"init_uniform"  # or any custom string
log_dir = f"runs/{run_name}"
writer = SummaryWriter(log_dir=log_dir)


In [8]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        self.image_paths = []
        self.labels = []

        class_names = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}

        for cls in class_names:
            cls_dir = os.path.join(root_dir, cls)
            for fname in os.listdir(cls_dir):
                if fname.lower().endswith((".png", ".jpg", ".jpeg")):
                    self.image_paths.append(os.path.join(cls_dir, fname))
                    self.labels.append(cls)

        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.labels)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        label = self.labels[idx]

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented["image"]

        return image, label

In [9]:
train_transform = A.Compose([
    A.Resize(64, 64),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(),
    ToTensorV2()
])


In [10]:
val_test_transform = A.Compose([
    A.Resize(64, 64),
    A.Normalize(),
    ToTensorV2()
])

In [11]:
# Paths
train_dir = "data/Split_smol/train"
val_dir = "data/Split_smol/val/"

In [12]:
train_dataset = CustomImageDataset(train_dir, transform=train_transform)
val_dataset   = CustomImageDataset(val_dir, transform=val_test_transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size)

In [13]:
class MLPClassifier(nn.Module):
    def __init__(self, input_size=64*64*3, num_classes=10):
        super().__init__()
        self.init_weights()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(input_size, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                #nn.init.kaiming_normal_(m.weight)
                #nn.init.xavier_uniform_
                #nn.init.kaiming_normal_
                nn.init.uniform_
                nn.init.zeros_(m.bias)

    def forward(self, x):
        return self.model(x)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(set(train_dataset.labels))
model = MLPClassifier(num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

In [15]:
# Entrenamiento y validación
def evaluate(model, loader, epoch=None, prefix="val"):
    log_classification_report(model, val_loader, writer, step=epoch, prefix="val")
    model.eval()
    correct, total, loss_sum = 0, 0, 0.0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for i, (images, labels) in enumerate(loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            loss_sum += loss.item()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Loguear imágenes del primer batch
            if i == 0 and epoch is not None:
                img_grid = vutils.make_grid(images[:8].cpu(), normalize=True)
                writer.add_image(f"{prefix}/images", img_grid, global_step=epoch)

    acc = 100.0 * correct / total
    avg_loss = loss_sum / len(loader)

    if epoch is not None:
        writer.add_scalar(f"{prefix}/loss", avg_loss, epoch)
        writer.add_scalar(f"{prefix}/accuracy", acc, epoch)

    return avg_loss, acc

In [16]:
# Loop de entrenamiento
n_epochs = 10
with mlflow.start_run(nested=True):
    # Log hiperparámetros
    mlflow.log_params({
        "model": "MLPClassifier",
        "input_size": 64*64*3,
        "batch_size": batch_size,
        "lr": 1e-3,
        "epochs": n_epochs,
        "optimizer": "Adam",
        "loss_fn": "CrossEntropyLoss",
        "train_dir": train_dir,
        "val_dir": val_dir,
    })
for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = 100.0 * correct / total
    val_loss, val_acc = evaluate(model, val_loader, epoch=epoch, prefix="val")

    print(f"Epoch {epoch+1}:")
    print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
    print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")

    writer.add_scalar("train/loss", train_loss, epoch)
    writer.add_scalar("train/accuracy", train_acc, epoch)

    # Log en MLflow
    mlflow.log_metrics({
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "val_loss": val_loss,
        "val_accuracy": val_acc
    }, step=epoch)

Epoch 1/10: 100%|██████████| 22/22 [00:13<00:00,  1.69it/s]


Epoch 1:
  Train Loss: 1.7203, Accuracy: 38.51%
  Val   Loss: 1.4556, Accuracy: 42.22%


Epoch 2/10: 100%|██████████| 22/22 [00:06<00:00,  3.31it/s]


Epoch 2:
  Train Loss: 1.3220, Accuracy: 50.72%
  Val   Loss: 1.2834, Accuracy: 51.11%


Epoch 3/10: 100%|██████████| 22/22 [00:06<00:00,  3.53it/s]


...

Epoch 8:
  Train Loss: 0.9263, Accuracy: 65.66%
  Val   Loss: 1.0589, Accuracy: 55.56%


Epoch 9/10: 100%|██████████| 22/22 [00:08<00:00,  2.52it/s]


Epoch 9:
  Train Loss: 0.8382, Accuracy: 69.11%
  Val   Loss: 1.1369, Accuracy: 56.67%


Epoch 10/10: 100%|██████████| 22/22 [00:09<00:00,  2.35it/s]


Epoch 10:
  Train Loss: 0.8469, Accuracy: 68.25%
  Val   Loss: 1.0493, Accuracy: 62.78%


In [17]:
# Guardar modelo
torch.save(model.state_dict(), "mlp_model.pth")
print("Modelo guardado como 'mlp_model.pth'")
mlflow.log_artifact("mlp_model.pth")
mlflow.pytorch.log_model(model, artifact_path="pytorch_model")
print("Modelo guardado como 'mlp_model.pth'")

2025/06/24 22:53:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Modelo guardado como 'mlp_model.pth'


2025/06/24 22:53:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo guardado como 'mlp_model.pth'


In [18]:
for name, param in model.named_parameters():
         writer.add_histogram(name, param, epoch)